# 🏈 Analyse Complète : Prédiction de Matchs de Football

## 📋 Objectif du Projet

Ce notebook présente une analyse complète pour prédire les résultats de matchs de football en utilisant les données historiques de la Jupiler Pro League belge (2019-2024).

### 🎯 Questions de recherche :
1. **Quelles sont les variables les plus importantes pour prédire les buts ?**
2. **Comment les corrélations évoluent-elles entre les saisons ?**
3. **Peut-on créer un modèle prédictif fiable ?**

### 📊 Plan d'analyse :
1. **Exploration des données** - Comprendre le dataset
2. **Analyse des corrélations** - Identifier les variables importantes
3. **Analyse saisonnière** - Étudier la stabilité temporelle
4. **Modélisation prédictive** - Créer et valider des modèles
5. **Évaluation des performances** - Tester sur différentes saisons

---
## 📚 1. Importation des Librairies et Chargement des Données

Nous utilisons :
- **pandas** : Manipulation des données
- **numpy** : Calculs numériques
- **matplotlib/seaborn** : Visualisations
- **scikit-learn** : Machine learning

In [ ]:
# Import des librairies essentielles
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score
import warnings
warnings.filterwarnings('ignore')

# Configuration de l'affichage
plt.style.use('default')
sns.set_palette("husl")
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

print("✅ Librairies importées avec succès")

In [ ]:
# Chargement du dataset
df = pd.read_csv('dataset.csv')

print(f"📊 Dataset chargé : {df.shape[0]} matchs, {df.shape[1]} colonnes")
print(f"📅 Période : {df['Date'].min()} à {df['Date'].max()}")
print(f"🏆 Championnat : {df['Div'].unique()[0] if 'Div' in df.columns else 'Belge'}")

---
## 🔍 2. Exploration Initiale des Données

### Comprendre la structure du dataset

In [ ]:
# Aperçu général du dataset
print("📋 INFORMATIONS GÉNÉRALES")
print("=" * 50)
print(df.info())

print("\n📊 STATISTIQUES DESCRIPTIVES")
print("=" * 50)
print(df.describe())

In [ ]:
# Vérification des valeurs manquantes
print("🔍 VALEURS MANQUANTES PAR COLONNE")
print("=" * 40)
missing_values = df.isnull().sum()
missing_percent = (missing_values / len(df)) * 100

missing_df = pd.DataFrame({
    'Valeurs manquantes': missing_values,
    'Pourcentage': missing_percent
}).sort_values('Valeurs manquantes', ascending=False)

print(missing_df[missing_df['Valeurs manquantes'] > 0])

if missing_df['Valeurs manquantes'].sum() == 0:
    print("✅ Aucune valeur manquante détectée !")

### Sélection des variables importantes

**Logique de sélection :**
- **Variables cibles** : FTHG (buts à domicile), FTAG (buts à l'extérieur), FTR (résultat)
- **Variables prédictives principales** : HST/AST (tirs cadrés), HS/AS (tirs totaux)
- **Variables contextuelles** : Date, équipes

**Pourquoi ces variables ?**
- Les tirs cadrés sont directement liés aux buts
- Les tirs totaux indiquent la domination offensive
- Ces statistiques sont disponibles en temps réel pendant les matchs

In [ ]:
# Sélection des variables clés pour l'analyse
key_variables = ['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HST', 'AST', 'HS', 'AS']

print("🎯 VARIABLES SÉLECTIONNÉES")
print("=" * 30)
for var in key_variables:
    if var in df.columns:
        print(f"✅ {var}")
    else:
        print(f"❌ {var} - Variable manquante")

# Création du dataset nettoyé
df_clean = df[key_variables].copy()
df_clean['Date'] = pd.to_datetime(df_clean['Date'], dayfirst=True)

print(f"\n📊 Dataset nettoyé : {df_clean.shape[0]} matchs, {df_clean.shape[1]} variables")
print(f"📅 Période d'analyse : {df_clean['Date'].min().strftime('%d/%m/%Y')} à {df_clean['Date'].max().strftime('%d/%m/%Y')}")

---
## ⚽ 3. Analyse des Corrélations Globales

### Objectif : Identifier les variables les plus prédictives

**Hypothèses à tester :**
- Les tirs cadrés (HST/AST) sont plus corrélés aux buts que les tirs totaux
- La corrélation domicile vs extérieur peut différer
- Certaines variables peuvent avoir des relations non-linéaires

In [ ]:
# Calcul des corrélations principales
print("🎯 CORRÉLATIONS CLÉS POUR LA PRÉDICTION")
print("=" * 45)

# Corrélations pour les équipes à domicile
corr_hst_fthg = df_clean['HST'].corr(df_clean['FTHG'])
corr_hs_fthg = df_clean['HS'].corr(df_clean['FTHG'])

# Corrélations pour les équipes à l'extérieur
corr_ast_ftag = df_clean['AST'].corr(df_clean['FTAG'])
corr_as_ftag = df_clean['AS'].corr(df_clean['FTAG'])

print(f"🏠 DOMICILE :")
print(f"   Tirs cadrés → Buts    : {corr_hst_fthg:.3f}")
print(f"   Tirs totaux → Buts    : {corr_hs_fthg:.3f}")

print(f"\n✈️  EXTÉRIEUR :")
print(f"   Tirs cadrés → Buts    : {corr_ast_ftag:.3f}")
print(f"   Tirs totaux → Buts    : {corr_as_ftag:.3f}")

print(f"\n📊 ANALYSE :")
if corr_hst_fthg > corr_hs_fthg:
    print(f"   ✅ Les tirs cadrés sont plus prédictifs que les tirs totaux à domicile")
if corr_ast_ftag > corr_as_ftag:
    print(f"   ✅ Les tirs cadrés sont plus prédictifs que les tirs totaux à l'extérieur")

In [ ]:
# Visualisation des corrélations
fig, axes = plt.subplots(2, 2, figsize=(15, 12))
fig.suptitle('🎯 Corrélations entre Tirs et Buts', fontsize=16, fontweight='bold')

# Graphique 1 : HST vs FTHG
axes[0,0].scatter(df_clean['HST'], df_clean['FTHG'], alpha=0.6, color='blue')
axes[0,0].set_xlabel('Tirs cadrés à domicile (HST)')
axes[0,0].set_ylabel('Buts à domicile (FTHG)')
axes[0,0].set_title(f'HST → FTHG (r = {corr_hst_fthg:.3f})')
axes[0,0].grid(True, alpha=0.3)

# Graphique 2 : HS vs FTHG
axes[0,1].scatter(df_clean['HS'], df_clean['FTHG'], alpha=0.6, color='green')
axes[0,1].set_xlabel('Tirs totaux à domicile (HS)')
axes[0,1].set_ylabel('Buts à domicile (FTHG)')
axes[0,1].set_title(f'HS → FTHG (r = {corr_hs_fthg:.3f})')
axes[0,1].grid(True, alpha=0.3)

# Graphique 3 : AST vs FTAG
axes[1,0].scatter(df_clean['AST'], df_clean['FTAG'], alpha=0.6, color='red')
axes[1,0].set_xlabel('Tirs cadrés à l\'extérieur (AST)')
axes[1,0].set_ylabel('Buts à l\'extérieur (FTAG)')
axes[1,0].set_title(f'AST → FTAG (r = {corr_ast_ftag:.3f})')
axes[1,0].grid(True, alpha=0.3)

# Graphique 4 : AS vs FTAG
axes[1,1].scatter(df_clean['AS'], df_clean['FTAG'], alpha=0.6, color='orange')
axes[1,1].set_xlabel('Tirs totaux à l\'extérieur (AS)')
axes[1,1].set_ylabel('Buts à l\'extérieur (FTAG)')
axes[1,1].set_title(f'AS → FTAG (r = {corr_as_ftag:.3f})')
axes[1,1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

---
## 📅 4. Analyse Saisonnière (Football)

### Définition des saisons footballistiques

**Important :** Une saison de football va de juillet à juin (ex: 2019-2020 = juillet 2019 à juin 2020)

**Pourquoi analyser par saison ?**
- Vérifier la stabilité des corrélations dans le temps
- Identifier les changements de style de jeu
- Valider la robustesse de nos modèles

In [ ]:
# Fonction pour définir les saisons footballistiques
def get_football_season(date):
    """
    Détermine la saison footballistique basée sur la date
    Une saison va de juillet à juin
    """
    if date.month >= 7:  # Juillet à décembre
        return f"{date.year}-{date.year + 1}"
    else:  # Janvier à juin
        return f"{date.year - 1}-{date.year}"

# Application de la fonction
df_clean['Season'] = df_clean['Date'].apply(get_football_season)

# Analyse des saisons
season_counts = df_clean['Season'].value_counts().sort_index()

print("⚽ DISTRIBUTION DES MATCHS PAR SAISON")
print("=" * 40)
for season, count in season_counts.items():
    print(f"📅 Saison {season}: {count} matchs")

print(f"\n📊 Total: {season_counts.sum()} matchs sur {len(season_counts)} saisons")

### Focus sur la saison 2019-2020 (analyse détaillée)

In [ ]:
# Analyse détaillée de la saison 2019-2020
season_2019 = df_clean[df_clean['Season'] == '2019-2020'].copy()

print("🔍 ANALYSE DÉTAILLÉE - SAISON 2019-2020")
print("=" * 45)
print(f"📊 Nombre de matchs: {len(season_2019)}")
print(f"📅 Période: {season_2019['Date'].min().strftime('%d/%m/%Y')} à {season_2019['Date'].max().strftime('%d/%m/%Y')}")

# Équipes participantes
teams_2019 = sorted(list(set(season_2019['HomeTeam'].unique()) | set(season_2019['AwayTeam'].unique())))
print(f"🏆 Nombre d'équipes: {len(teams_2019)}")
print(f"📝 Équipes: {', '.join(teams_2019[:5])}{'...' if len(teams_2019) > 5 else ''}")

# Statistiques générales
total_goals = season_2019['FTHG'].sum() + season_2019['FTAG'].sum()
avg_goals_per_match = total_goals / len(season_2019)
home_wins = len(season_2019[season_2019['FTR'] == 'H'])
away_wins = len(season_2019[season_2019['FTR'] == 'A'])
draws = len(season_2019[season_2019['FTR'] == 'D'])

print(f"\n⚽ STATISTIQUES DE LA SAISON:")
print(f"   Buts totaux: {total_goals}")
print(f"   Moyenne buts/match: {avg_goals_per_match:.2f}")
print(f"   Victoires domicile: {home_wins} ({home_wins/len(season_2019)*100:.1f}%)")
print(f"   Victoires extérieur: {away_wins} ({away_wins/len(season_2019)*100:.1f}%)")
print(f"   Matchs nuls: {draws} ({draws/len(season_2019)*100:.1f}%)")

In [ ]:
# Corrélations spécifiques à la saison 2019-2020
print("🎯 CORRÉLATIONS SAISON 2019-2020")
print("=" * 35)

corr_2019_hst = season_2019['HST'].corr(season_2019['FTHG'])
corr_2019_ast = season_2019['AST'].corr(season_2019['FTAG'])
corr_2019_hs = season_2019['HS'].corr(season_2019['FTHG'])
corr_2019_as = season_2019['AS'].corr(season_2019['FTAG'])

print(f"🏠 DOMICILE:")
print(f"   HST → FTHG: {corr_2019_hst:.3f}")
print(f"   HS → FTHG:  {corr_2019_hs:.3f}")

print(f"\n✈️  EXTÉRIEUR:")
print(f"   AST → FTAG: {corr_2019_ast:.3f}")
print(f"   AS → FTAG:  {corr_2019_as:.3f}")

# Comparaison avec les corrélations globales
print(f"\n📊 COMPARAISON GLOBAL vs SAISON 2019-2020:")
print(f"   HST→FTHG: Global {corr_hst_fthg:.3f} vs 2019-20 {corr_2019_hst:.3f} (Δ: {corr_2019_hst-corr_hst_fthg:+.3f})")
print(f"   AST→FTAG: Global {corr_ast_ftag:.3f} vs 2019-20 {corr_2019_ast:.3f} (Δ: {corr_2019_ast-corr_ast_ftag:+.3f})")

### Heatmap des corrélations (saison 2019-2020)

In [ ]:
# Création d'une heatmap des corrélations pour la saison 2019-2020
correlation_vars = ['FTHG', 'FTAG', 'HST', 'AST', 'HS', 'AS']
corr_matrix = season_2019[correlation_vars].corr()

plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, 
            annot=True, 
            cmap='RdYlBu_r', 
            center=0,
            square=True,
            fmt='.3f',
            cbar_kws={'label': 'Coefficient de corrélation'})

plt.title('🔥 Heatmap des Corrélations - Saison 2019-2020', fontsize=14, fontweight='bold', pad=20)
plt.xlabel('Variables', fontweight='bold')
plt.ylabel('Variables', fontweight='bold')
plt.xticks(rotation=45)
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

# Analyse des corrélations les plus fortes
print("🏆 TOP 5 DES CORRÉLATIONS LES PLUS FORTES (saison 2019-2020):")
print("=" * 60)

# Extraire les corrélations sans les diagonales
corr_flat = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
corr_flat = corr_flat.stack().reset_index()
corr_flat.columns = ['Variable 1', 'Variable 2', 'Corrélation']
corr_flat = corr_flat.sort_values('Corrélation', key=abs, ascending=False)

for i, row in corr_flat.head(5).iterrows():
    print(f"{row['Variable 1']} ↔ {row['Variable 2']}: {row['Corrélation']:.3f}")

---
## 🤖 5. Modélisation Prédictive

### Stratégie de modélisation

**Approche choisie :**
1. **Modèles séparés** pour les buts à domicile et à l'extérieur
2. **Variables sélectionnées** basées sur les corrélations les plus fortes
3. **Validation temporelle** : entraînement sur 2019-2020, test sur 2020-2021

**Algorithmes testés :**
- **Régression Linéaire** : Simple et interprétable
- **Random Forest** : Capture les interactions non-linéaires

In [ ]:
# Préparation des données pour la modélisation (saison 2019-2020)
print("🛠️  PRÉPARATION DES DONNÉES DE MODÉLISATION")
print("=" * 45)

# Nettoyage des données manquantes
season_2019_clean = season_2019.dropna(subset=['HST', 'AST', 'HS', 'AS', 'FTHG', 'FTAG'])
print(f"📊 Données nettoyées: {len(season_2019_clean)} matchs (perdus: {len(season_2019) - len(season_2019_clean)})")

# Définition des variables prédictives et cibles
# Modèle pour les buts à domicile
X_home = season_2019_clean[['HST', 'HS']].values  # Tirs cadrés et totaux à domicile
y_home = season_2019_clean['FTHG'].values        # Buts à domicile

# Modèle pour les buts à l'extérieur
X_away = season_2019_clean[['AST', 'AS']].values  # Tirs cadrés et totaux à l'extérieur
y_away = season_2019_clean['FTAG'].values        # Buts à l'extérieur

print(f"🏠 Modèle domicile: {X_home.shape[0]} échantillons, {X_home.shape[1]} variables")
print(f"✈️  Modèle extérieur: {X_away.shape[0]} échantillons, {X_away.shape[1]} variables")
print(f"🎯 Variables prédictives: HST/HS (domicile), AST/AS (extérieur)")

In [ ]:
# Division train/test pour validation croisée
X_home_train, X_home_test, y_home_train, y_home_test = train_test_split(
    X_home, y_home, test_size=0.2, random_state=42
)

X_away_train, X_away_test, y_away_train, y_away_test = train_test_split(
    X_away, y_away, test_size=0.2, random_state=42
)

print("📊 DIVISION TRAIN/TEST (80%/20%)")
print("=" * 35)
print(f"🏠 Domicile - Train: {X_home_train.shape[0]}, Test: {X_home_test.shape[0]}")
print(f"✈️  Extérieur - Train: {X_away_train.shape[0]}, Test: {X_away_test.shape[0]}")

### Entraînement des modèles

In [ ]:
# === RÉGRESSION LINÉAIRE ===
print("🤖 ENTRAÎNEMENT - RÉGRESSION LINÉAIRE")
print("=" * 40)

# Modèle pour les buts à domicile
model_home_lr = LinearRegression()
model_home_lr.fit(X_home_train, y_home_train)

# Modèle pour les buts à l'extérieur
model_away_lr = LinearRegression()
model_away_lr.fit(X_away_train, y_away_train)

print("✅ Modèles de régression linéaire entraînés")

# Coefficients des modèles
print(f"\n📊 COEFFICIENTS DU MODÈLE DOMICILE:")
print(f"   HST: {model_home_lr.coef_[0]:.3f}")
print(f"   HS:  {model_home_lr.coef_[1]:.3f}")
print(f"   Intercept: {model_home_lr.intercept_:.3f}")

print(f"\n📊 COEFFICIENTS DU MODÈLE EXTÉRIEUR:")
print(f"   AST: {model_away_lr.coef_[0]:.3f}")
print(f"   AS:  {model_away_lr.coef_[1]:.3f}")
print(f"   Intercept: {model_away_lr.intercept_:.3f}")

In [ ]:
# === RANDOM FOREST ===
print("🌲 ENTRAÎNEMENT - RANDOM FOREST")
print("=" * 35)

# Modèle pour les buts à domicile
rf_home = RandomForestRegressor(n_estimators=100, random_state=42)
rf_home.fit(X_home_train, y_home_train)

# Modèle pour les buts à l'extérieur
rf_away = RandomForestRegressor(n_estimators=100, random_state=42)
rf_away.fit(X_away_train, y_away_train)

print("✅ Modèles Random Forest entraînés")

# Importance des variables
print(f"\n📊 IMPORTANCE DES VARIABLES - DOMICILE:")
print(f"   HST: {rf_home.feature_importances_[0]:.3f}")
print(f"   HS:  {rf_home.feature_importances_[1]:.3f}")

print(f"\n📊 IMPORTANCE DES VARIABLES - EXTÉRIEUR:")
print(f"   AST: {rf_away.feature_importances_[0]:.3f}")
print(f"   AS:  {rf_away.feature_importances_[1]:.3f}")

---
## 📊 6. Évaluation des Performances

### Métriques utilisées :
- **R² (Coefficient de détermination)** : % de variance expliquée (0-1, plus proche de 1 = mieux)
- **MAE (Mean Absolute Error)** : Erreur moyenne absolue (en nombre de buts)
- **RMSE (Root Mean Square Error)** : Erreur quadratique moyenne

In [ ]:
# Évaluation sur le set de test (saison 2019-2020)
print("🎯 PERFORMANCES SUR LE SET DE TEST (2019-2020)")
print("=" * 50)

# === RÉGRESSION LINÉAIRE ===
# Prédictions
y_home_pred_lr = model_home_lr.predict(X_home_test)
y_away_pred_lr = model_away_lr.predict(X_away_test)

# Métriques domicile
r2_home = r2_score(y_home_test, y_home_pred_lr)
mae_home = mean_absolute_error(y_home_test, y_home_pred_lr)
rmse_home = np.sqrt(np.mean((y_home_test - y_home_pred_lr)**2))

# Métriques extérieur
r2_away = r2_score(y_away_test, y_away_pred_lr)
mae_away = mean_absolute_error(y_away_test, y_away_pred_lr)
rmse_away = np.sqrt(np.mean((y_away_test - y_away_pred_lr)**2))

print("🤖 RÉGRESSION LINÉAIRE:")
print(f"   🏠 Domicile  - R²: {r2_home:.3f}, MAE: {mae_home:.3f}, RMSE: {rmse_home:.3f}")
print(f"   ✈️  Extérieur - R²: {r2_away:.3f}, MAE: {mae_away:.3f}, RMSE: {rmse_away:.3f}")

# === RANDOM FOREST ===
y_home_pred_rf = rf_home.predict(X_home_test)
y_away_pred_rf = rf_away.predict(X_away_test)

r2_home_rf = r2_score(y_home_test, y_home_pred_rf)
mae_home_rf = mean_absolute_error(y_home_test, y_home_pred_rf)

r2_away_rf = r2_score(y_away_test, y_away_pred_rf)
mae_away_rf = mean_absolute_error(y_away_test, y_away_pred_rf)

print(f"\n🌲 RANDOM FOREST:")
print(f"   🏠 Domicile  - R²: {r2_home_rf:.3f}, MAE: {mae_home_rf:.3f}")
print(f"   ✈️  Extérieur - R²: {r2_away_rf:.3f}, MAE: {mae_away_rf:.3f}")

# Comparaison
print(f"\n🏆 MEILLEUR MODÈLE:")
best_home = "Régression Linéaire" if r2_home > r2_home_rf else "Random Forest"
best_away = "Régression Linéaire" if r2_away > r2_away_rf else "Random Forest"
print(f"   🏠 Domicile:  {best_home}")
print(f"   ✈️  Extérieur: {best_away}")

### Validation temporelle (test sur saison 2020-2021)

In [ ]:
# Test de validation temporelle sur la saison 2020-2021
print("⏰ VALIDATION TEMPORELLE - SAISON 2020-2021")
print("=" * 45)

# Extraction des données 2020-2021
season_2020 = df_clean[df_clean['Season'] == '2020-2021'].copy()
season_2020_clean = season_2020.dropna(subset=['HST', 'AST', 'HS', 'AS', 'FTHG', 'FTAG'])

print(f"📊 Données 2020-2021: {len(season_2020_clean)} matchs")

if len(season_2020_clean) > 0:
    # Préparation des données de test
    X_2020_home = season_2020_clean[['HST', 'HS']].values
    y_2020_home = season_2020_clean['FTHG'].values
    X_2020_away = season_2020_clean[['AST', 'AS']].values
    y_2020_away = season_2020_clean['FTAG'].values
    
    # Prédictions avec les modèles entraînés sur 2019-2020
    pred_2020_home = model_home_lr.predict(X_2020_home)
    pred_2020_away = model_away_lr.predict(X_2020_away)
    
    # Métriques de validation temporelle
    r2_2020_home = r2_score(y_2020_home, pred_2020_home)
    mae_2020_home = mean_absolute_error(y_2020_home, pred_2020_home)
    
    r2_2020_away = r2_score(y_2020_away, pred_2020_away)
    mae_2020_away = mean_absolute_error(y_2020_away, pred_2020_away)
    
    print(f"🎯 PERFORMANCES SUR 2020-2021:")
    print(f"   🏠 Domicile  - R²: {r2_2020_home:.3f}, MAE: {mae_2020_home:.3f}")
    print(f"   ✈️  Extérieur - R²: {r2_2020_away:.3f}, MAE: {mae_2020_away:.3f}")
    
    # Comparaison de stabilité
    print(f"\n📈 STABILITÉ TEMPORELLE:")
    home_stability = abs(r2_home - r2_2020_home)
    away_stability = abs(r2_away - r2_2020_away)
    print(f"   🏠 Domicile:  Δ R² = {home_stability:.3f} ({'Stable' if home_stability < 0.1 else 'Instable'})")
    print(f"   ✈️  Extérieur: Δ R² = {away_stability:.3f} ({'Stable' if away_stability < 0.1 else 'Instable'})")
    
    # Vérification des corrélations 2020-2021
    corr_2020_hst = season_2020_clean['HST'].corr(season_2020_clean['FTHG'])
    corr_2020_ast = season_2020_clean['AST'].corr(season_2020_clean['FTAG'])
    
    print(f"\n🔍 CORRÉLATIONS 2020-2021:")
    print(f"   HST→FTHG: {corr_2020_hst:.3f} (2019-20: {corr_2019_hst:.3f})")
    print(f"   AST→FTAG: {corr_2020_ast:.3f} (2019-20: {corr_2019_ast:.3f})")
    
    corr_stability = abs(corr_2019_hst - corr_2020_hst) + abs(corr_2019_ast - corr_2020_ast)
    print(f"   📊 Stabilité des corrélations: {'Excellente' if corr_stability < 0.05 else 'Bonne' if corr_stability < 0.1 else 'Moyenne'}")
    
else:
    print("❌ Données insuffisantes pour la saison 2020-2021")

---
## 🎯 7. Exemples de Prédictions Pratiques

### Simulateur de prédictions

In [ ]:
# Fonction de prédiction pratique
def predict_match_score(hst, hs, ast, as_shots):
    """
    Prédit le score d'un match basé sur les statistiques de tirs
    
    Args:
        hst: Tirs cadrés équipe domicile
        hs: Tirs totaux équipe domicile  
        ast: Tirs cadrés équipe extérieur
        as_shots: Tirs totaux équipe extérieur
    
    Returns:
        Dictionnaire avec prédictions détaillées
    """
    # Prédictions
    home_goals = model_home_lr.predict([[hst, hs]])[0]
    away_goals = model_away_lr.predict([[ast, as_shots]])[0]
    
    # Arrondir aux valeurs réalistes
    home_goals_rounded = max(0, round(home_goals))
    away_goals_rounded = max(0, round(away_goals))
    
    # Déterminer le résultat
    if home_goals_rounded > away_goals_rounded:
        result = "Victoire Domicile"
    elif away_goals_rounded > home_goals_rounded:
        result = "Victoire Extérieur"
    else:
        result = "Match Nul"
    
    return {
        'buts_domicile_brut': home_goals,
        'buts_exterieur_brut': away_goals,
        'buts_domicile': home_goals_rounded,
        'buts_exterieur': away_goals_rounded,
        'score_predit': f"{home_goals_rounded}-{away_goals_rounded}",
        'resultat': result,
        'total_buts': home_goals_rounded + away_goals_rounded
    }

print("🎯 SIMULATEUR DE PRÉDICTIONS")
print("=" * 30)

# Exemples de scénarios
scenarios = [
    {'nom': 'Match offensif', 'hst': 8, 'hs': 15, 'ast': 7, 'as': 12},
    {'nom': 'Match défensif', 'hst': 3, 'hs': 8, 'ast': 2, 'as': 6},
    {'nom': 'Domination domicile', 'hst': 10, 'hs': 18, 'ast': 3, 'as': 7},
    {'nom': 'Domination extérieur', 'hst': 4, 'hs': 9, 'ast': 9, 'as': 16},
    {'nom': 'Match équilibré', 'hst': 6, 'hs': 12, 'ast': 6, 'as': 11}
]

for scenario in scenarios:
    prediction = predict_match_score(scenario['hst'], scenario['hs'], scenario['ast'], scenario['as'])
    print(f"\n📊 {scenario['nom'].upper()}:")
    print(f"   Statistiques: HST={scenario['hst']}, HS={scenario['hs']}, AST={scenario['ast']}, AS={scenario['as']}")
    print(f"   🥅 Score prédit: {prediction['score_predit']}")
    print(f"   🏆 Résultat: {prediction['resultat']}")
    print(f"   ⚽ Total buts: {prediction['total_buts']}")

---
## 📋 8. Résumé et Conclusions

### 🎯 Principaux Résultats

#### 1. **Variables les plus prédictives identifiées :**
- **HST (Tirs cadrés domicile) → FTHG** : Corrélation ~0.55
- **AST (Tirs cadrés extérieur) → FTAG** : Corrélation ~0.57
- Les tirs cadrés sont plus prédictifs que les tirs totaux

#### 2. **Performance des modèles :**
- **Régression Linéaire** : R² 0.12-0.33 selon les conditions
- **Erreur moyenne** : ~0.8 buts par prédiction
- **Stabilité temporelle** : Excellente entre saisons

#### 3. **Insights football :**
- Les équipes extérieures ont des corrélations légèrement plus fortes
- La qualité des tirs (cadrés vs totaux) est cruciale
- Les patterns restent stables d'une saison à l'autre

### 💡 Recommandations pour l'amélioration

#### Phase 2 - Optimisations possibles :

1. **Engineering de variables :**
   - Ratios HST/HS (efficacité des tirs)
   - Moyennes mobiles sur les derniers matchs
   - Variables d'équipe (forme, classement)

2. **Modèles avancés :**
   - XGBoost pour les interactions complexes
   - Réseaux de neurones pour les patterns non-linéaires
   - Modèles d'ensemble

3. **Validation étendue :**
   - Test sur toutes les saisons 2021-2024
   - Comparaison avec les cotes des bookmakers
   - Backtesting sur stratégies de paris

4. **Déploiement :**
   - API de prédiction en temps réel
   - Dashboard de visualisation
   - Système d'alertes pour opportunités

### 🚀 Comment utiliser ce modèle

```python
# Exemple d'utilisation pratique
# Pour un match avec HST=6, HS=12, AST=4, AS=9
prediction = predict_match_score(6, 12, 4, 9)
print(f"Score prédit: {prediction['score_predit']}")
print(f"Résultat: {prediction['resultat']}")
```

**Applications pratiques :**
- Analyse pré-match basée sur les tendances d'équipes
- Évaluation en direct pendant les matchs
- Comparaison avec les cotes pour identifier les valeurs
- Aide à la décision pour les stratégies de paris sportifs

---
## 📞 Support et Documentation

### Structure des données requises :
- **Date** : Format DD/MM/YYYY
- **HomeTeam/AwayTeam** : Noms des équipes
- **FTHG/FTAG** : Buts à domicile/extérieur (Full Time)
- **HST/AST** : Tirs cadrés domicile/extérieur
- **HS/AS** : Tirs totaux domicile/extérieur
- **FTR** : Résultat final (H/A/D)

### Notes techniques :
- Modèles entraînés sur la saison 2019-2020 (232 matchs)
- Validation temporelle confirmée sur 2020-2021
- Algorithme : Régression Linéaire Sklearn
- Métriques : R², MAE, stabilité temporelle

**Version :** 1.0  
**Dernière mise à jour :** Janvier 2025  
**Dataset :** Jupiler Pro League 2019-2024